In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from state import get_state, queue
import os
import traci

In [4]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

In [5]:
import tensorflow as tf
import numpy as np
from collections import deque

In [17]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation='relu', input_shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(n_outputs, activation= 'linear')])

/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
target = tf.keras.models.clone_model(model_action)  # clone the model's architecture
target.set_weights(model_action.get_weights())  # copy the weights

In [19]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [20]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [21]:
batch_size = 32
discount_factor = 0.5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_fn = tf.keras.losses.MeanSquaredError()


def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
        #################### CHANGED SECTION ####################
    next_Q_values = model_action.predict(next_states, verbose=0)  # ≠ target.predict()
    best_next_actions = next_Q_values.argmax(axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    max_next_Q_values = (target.predict(next_states, verbose=0) * next_mask
                        ).sum(axis=1)
    #########################################################
    next_Q_values = target.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [22]:
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"

replay_buffer = deque(maxlen=10000)


sumoCmd = [sumo_bin, "-c",simulConfig,'--start']
print(sumoCmd)
for episode in range(80):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    for step in range(130000): ## TO CHANGED
        epsilon = max(1 - episode / 80, 0.01)

        if step%2000 == 0:
            #######################################################################
            next_state = np.array(get_state(lane_ids))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))
            #######################################################################

            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)

        traci.simulationStep()

    if episode%5==0:
        target.set_weights(model_action.get_weights())
    print(f'episode : {episode}')
    traci.close()


['/Users/arseneclaustre/sumo/bin/sumo', '-c', 'Traditional_traffic/traditional_traffic.sumo.cfg', '--start']
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1334616ms, vehicles TOT 0 ACT 0 BUF 0)                
 Retrying in 1 seconds


Step #220.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 176 ACT 55 BUF 4)                  

Step #980.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 755 ACT 100 BUF 1)                 

Step #1160.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 887 ACT 110 BUF 1)                

episode : 0
Step #1300.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 924 ACT 63 BUF 0)                 
 Retrying in 1 seconds


...

state : [ 0  2 11  0  0  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2 19  2  2  4  4  2  0  0  3  0  0  0  0  4  6  6  0  0  0  0  1  0]
Q_values : [-6.2944865  -4.2335987  -0.68823195 -5.344147  ]
state : [ 0  0 26  0  1  6  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
  2  3 28  2  4  7  5  2  1  3  6  1  2  1  0  3  1  0  0  1  0  0  0  0]
Q_values : [-1.900111    9.288179    8.25634     0.21142888]
state : [ 0  2  0  0  3  8  0  2  0  0  9  2  0  0  0  0  0  0  0  0  0  0  0  0
  1  2 14  2  5 10  6  2  0  1 13  2  1  0  0  2  0 13  1  0  0  0  0  1]
Q_values : [-0.02622199 -1.5517279   8.246913    2.812395  ]


state : [ 0  2 10  0  0  9  0  2  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0
  0  2 17  0  0 10  4  2  0  0  4  2  2  0  0  4  6  7  0  0  0  0  3  2]
Q_values : [ 3.0694218  6.1808133  4.351209  10.343629 ]
state : [ 0  2 17  0  0  0  0  2  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  3 20  2  2  2  4  2  0  1  7  0  0  0  1  3  2  0  0  0  8  0  0  0]
Q_values : [-3.3385766  9.158918   6.0491657 -0.6380615]
state : [ 0  2 10  0  0  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2 16  2  2  4  4  2  0  0  3  0  0  0  0  4  7  7  0  0  0  0  1  0]
Q_values : [-0.17995477  4.383688   -0.6644964  -0.29824632]


state : [0 2 0 0 2 4 0 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 4 3 7 4 3 4 6 3 0 4 6 0 2
 0 0 3 1 9 0 0 0 0 0 0]
Q_values : [-5.333116    0.83052635 -0.3487239  -1.0713077 ]
state : [ 0  2  0  0  2  4  0  2  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0
  2  4  5  1  4  6  5  4  2  2 10  2  0  0  0  2  0  9  0  0  0  0  0  0]
Q_values : [-6.866949   -0.71996534 -0.75597525 -1.8729403 ]
state : [ 0  4  0  0  4  6  0  4  0  0  9  2  0  0  0  0  0  0  0  0  0  0  0  0
  0  4  3  2  7  7  4  4  0  0 13  2  2  0  0  4  0  3  2  0  0  2  0  2]
Q_values : [ 0.96347    5.0648303 10.086792   8.916183 ]


Interrupt signal received, trying to exit gracefully.T 419 ACT 53 BUF 1)                  


KeyboardInterrupt: 

In [ ]:
model_action.

In [26]:
sumoCmd = [sumo_gui_bin, "-c",simulConfig,"--start", "--no-warnings"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=1
wating_times= np.ones(shape=(24))
for step in range(130000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state = np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.5612404  1.1188273 -1.5535202  0.8271354]


FatalTraCIError: Connection closed by SUMO.